In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download de_core_news_sm
!pip install emoji
!pip install langdetect
!pip install transformers
!pip install ekphrasis
!pip install -U datasets

There are some helper functions written in some external python files. We load these python files directly into colab.

In [1]:
from google.colab import files
files.upload()

Saving inferencing.py to inferencing.py
Saving model_params.py to model_params.py
Saving preprocessing.py to preprocessing.py
Saving training.py to training.py


{'inferencing.py': b'import hashlib \nimport pymongo\nimport torch\nimport math\n\nfrom pymongo_ssh import MongoSession\n\n\n# connect to database\ndef connect_to_database(key, host=\'138.246.233.159\'):\n    session = MongoSession(\n    host=host,\n    port=22,\n    user=\'ubuntu\',\n    key=key,\n    uri=\'mongodb://127.0.0.1:27017\')\n    db = session.connection[\'telegram\']\n    \n    db_channels = db.channels\n    db_messages = db.messages\n    db_errors = db.errors\n    db_users = db.users\n    db_entities = db.entities\n    \n    return db_channels, db_messages, db_errors, db_users, db_entities\n\ndef hashContent(field1,field2,field3):\n    to_hash = field1 if field1 is not None else \'\'\n    if field2 is not None:\n        to_hash = to_hash + field2\n    if field3 is not None:\n        to_hash = to_hash + field3\n    result = hashlib.md5(to_hash.encode()) \n    return result.hexdigest()\n\ndef predict(text,tokenizer,model):\n    inputs =  tokenizer(text, return_tensors="pt", 

In [5]:
import torch
from torch import nn
import json
import nltk
nltk.download('stopwords')
import preprocessing as pp
from training import *
import model_params as mp

from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments,BertForSequenceClassification, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from datasets import load_dataset
import numpy as np
from sklearn.utils import class_weight

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
selected_dataset = 'covid_2021'
MODEL_NAME = 'dbmdz/bert-base-german-cased'
#MODEL_NAME = 'german-nlp-group/electra-base-german-uncased'
#MODEL_NAME = 'deepset/gbert-base'

In [6]:
# load model and tokenizer
num_classes = len(mp.model_params[selected_dataset]['mapping'])
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = num_classes)

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initi

Before training, we do some preprocessing on the data (e.g. masking URLs)

In [9]:
# load data
dataset = load_dataset('csv', data_files={'train': mp.model_params[selected_dataset]['train'],
                                                 'validation': mp.model_params[selected_dataset]['validation'],
                                                 'test': mp.model_params[selected_dataset]['test']}, delimiter='\t')

Using custom data configuration default-b2022dffb8bbb985


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b2022dffb8bbb985/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


In [10]:
def preprocess(row):
    row['text'] = pp.prepare_single_text(row['text'])
    row[mp.model_params[selected_dataset]['label']] = mp.model_params[selected_dataset]['mapping'][row[mp.model_params[selected_dataset]['label']]]
    return row

In [11]:
# preprocess data
dataset['train'] = dataset['train'].map(preprocess)
dataset['validation'] = dataset['validation'].map(preprocess)
dataset['test'] = dataset['test'].map(preprocess)

# tokenize data
train_encodings = tokenizer(dataset['train']['text'], truncation=True, padding=True)
val_encodings = tokenizer(dataset['validation']['text'], truncation=True, padding=True)
test_encodings = tokenizer(dataset['test']['text'], truncation=True, padding=True)

train_dataset = HateDataset(train_encodings, dataset['train'][mp.model_params[selected_dataset]['label']])
val_dataset = HateDataset(val_encodings, dataset['validation'][mp.model_params[selected_dataset]['label']])
test_dataset = HateDataset(test_encodings, dataset['test'][mp.model_params[selected_dataset]['label']])

Some of the training datasets are really imbalanced. To tackle this issue we used a custom Trainer class that implements class weights.

In [12]:
# overwrite Trainer to add class weights
class MyTrainer(Trainer):
    """
    Overwritten Trainer class
    - Able to apply class_weights for imbalanced training. Recommended by the FARM framework: https://github.com/deepset-ai/FARM/tree/master#3-imbalanced-classes
    """

    def set_class_weights(self, class_weights, mode=True):
        self.use_class_weights = mode
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.class_weights = class_weights.to(self.device)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        if self.use_class_weights:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        else:
            loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, model.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [13]:
# compute class weights
train_labels = dataset['train'][mp.model_params[selected_dataset]['label']]

class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels), train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6450, 2.2242])


Define some parameters for the training...

In [14]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=8,              # total # of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    evaluation_strategy='steps',
    eval_steps=300,
    load_best_model_at_end=True ,
    metric_for_best_model='eval_f1_macro',
)

trainer = MyTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics if num_classes == 2 else compute_metrics_multiclass,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

In [15]:
trainer.set_class_weights(class_weights=class_weights, mode=True)
trainer.train()

***** Running training *****
  Num examples = 3532
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3536


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,F1 Micro,F1 Macro,F1 Weighted
300,No log,0.713760,0.867123,0.720779,0.672727,0.695925,0.867123,0.805456,0.865473
600,0.556300,0.715928,0.858904,0.734848,0.587879,0.653199,0.858904,0.782317,0.853067
900,0.556300,1.006642,0.867123,0.761538,0.600000,0.671186,0.867123,0.793962,0.861237
1200,0.466300,0.949760,0.883562,0.770270,0.690909,0.728435,0.883562,0.827164,0.881262
1500,0.275700,1.042302,0.875342,0.728395,0.715152,0.721713,0.875342,0.820697,0.874936
1800,0.275700,1.485267,0.867123,0.783333,0.569697,0.659649,0.867123,0.788548,0.859177
2100,0.096000,1.271747,0.860274,0.672131,0.745455,0.706897,0.860274,0.807585,0.862757
2400,0.096000,1.523269,0.871233,0.748252,0.648485,0.694805,0.871233,0.806604,0.867864


***** Running Evaluation *****
  Num examples = 730
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-300
Configuration saved in ./results/checkpoint-300/config.json
Model weights saved in ./results/checkpoint-300/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 730
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-600
Configuration saved in ./results/checkpoint-600/config.json
Model weights saved in ./results/checkpoint-600/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 730
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-900
Configuration saved in ./results/checkpoint-900/config.json
Model weights saved in ./results/checkpoint-900/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 730
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-1200
Configuration saved in ./results/checkpoint-1200/config.json
Model weights saved in ./results/checkpoint-1200/pytorch_model.bin
*****

TrainOutput(global_step=2400, training_loss=0.2949590750535329, metrics={'train_runtime': 603.6836, 'train_samples_per_second': 46.806, 'train_steps_per_second': 5.857, 'total_flos': 3213266219489760.0, 'train_loss': 0.2949590750535329, 'epoch': 5.43})

Now we evaluate the model on the test set.

In [16]:
results_it = trainer.predict(test_dataset) 
print(results_it.metrics)

***** Running Prediction *****
  Num examples = 698
  Batch size = 32


{'test_loss': 1.1396424770355225, 'test_accuracy': 0.8567335243553008, 'test_precision': 0.6825396825396826, 'test_recall': 0.589041095890411, 'test_f1': 0.6323529411764707, 'test_f1_micro': 0.8567335243553008, 'test_f1_macro': 0.7716924848231108, 'test_f1_weighted': 0.8527409872880619, 'test_runtime': 4.471, 'test_samples_per_second': 156.118, 'test_steps_per_second': 4.921}


We store the model such that we can later download the best model for this dataset.

In [17]:
model.save_pretrained(f"./models/{selected_dataset}/")
results_to_save = results_it.metrics
results_to_save['model'] = MODEL_NAME
results_to_save['dataset'] = selected_dataset
results_to_save['train_size'] = len(train_dataset)
results_to_save['validation_size'] = len(val_dataset)
results_to_save['test_size'] = len(test_dataset)
with open(f'./models/{selected_dataset}/results.json', 'w+') as file:
     file.write(json.dumps(results_to_save))

Configuration saved in ./models/covid_2021/config.json
Model weights saved in ./models/covid_2021/pytorch_model.bin
